# 01 - Data Preprocessing and Cleaning

This notebook handles:
- Loading the raw dataset
- Data cleaning and validation
- Handling missing values
- Data type conversions
- Saving processed data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Load Dataset

In [2]:
# Load the dataset
df = pd.read_csv('../dataset/Steel_industry_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

Dataset shape: (35040, 11)

First few rows:


,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


## Initial Data Inspection

In [3]:
# Display dataset information
print("Dataset Info:")
df.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  35040 non-null  object 
 1   Usage_kWh                             35040 non-null  float64
 2   Lagging_Current_Reactive.Power_kVarh  35040 non-null  float64
 3   Leading_Current_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2(tCO2)                             35040 non-null  float64
 5   Lagging_Current_Power_Factor          35040 non-null  float64
 6   Leading_Current_Power_Factor          35040 non-null  float64
 7   NSM                                   35040 non-null  int64  
 8   WeekStatus                            35040 non-null  object 
 9   Day_of_week                           35040 non-null  object 
 10  Load_Type                             35040 non-null  object 
dtypes

In [4]:
# Check for missing values
print("Missing Values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

if missing_values.sum() == 0:
    print("\nNo missing values found!")

Missing Values:
Series([], dtype: int64)

No missing values found!


In [5]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [6]:
# Statistical summary
print("Statistical Summary:")
df.describe()

Statistical Summary:


,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,27.386892,13.035384,3.870949,0.011524,80.578056,84.367870,42750.000000
std,33.444380,16.306000,7.424463,0.016151,18.921322,30.456535,24940.534317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000,0.000000,63.320000,99.700000,21375.000000
50%,4.570000,5.000000,0.000000,0.000000,87.960000,100.000000,42750.000000
75%,51.237500,22.640000,2.090000,0.020000,99.022500,100.000000,64125.000000
max,157.180000,96.910000,27.760000,0.070000,100.000000,100.000000,85500.000000


## Data Type Conversions

In [7]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

print("Date column converted to datetime")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

Date column converted to datetime
Date range: 2018-01-01 00:00:00 to 2018-12-31 23:45:00


In [8]:
# Check categorical columns
categorical_cols = ['WeekStatus', 'Day_of_week', 'Load_Type']

for col in categorical_cols:
    print(f"\n{col} unique values:")
    print(df[col].value_counts())


WeekStatus unique values:
WeekStatus
Weekday    25056
Weekend     9984
Name: count, dtype: int64

Day_of_week unique values:
Day_of_week
Monday       5088
Tuesday      4992
Wednesday    4992
Thursday     4992
Friday       4992
Saturday     4992
Sunday       4992
Name: count, dtype: int64

Load_Type unique values:
Load_Type
Light_Load      18072
Medium_Load      9696
Maximum_Load     7272
Name: count, dtype: int64


## Data Validation

In [9]:
# Check for negative values in features that should be positive
numeric_cols = ['Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 
                'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 
                'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM']

for col in numeric_cols:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f"{col}: {negative_count} negative values found")

print("\nValidation complete!")


Validation complete!


In [10]:
# Check for outliers in target variable (Usage_kWh)
Q1 = df['Usage_kWh'].quantile(0.25)
Q3 = df['Usage_kWh'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[(df['Usage_kWh'] < Q1 - 1.5*IQR) | (df['Usage_kWh'] > Q3 + 1.5*IQR)]
print(f"Number of outliers in Usage_kWh: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
print(f"\nUsage_kWh statistics:")
print(f"Min: {df['Usage_kWh'].min():.2f}")
print(f"Max: {df['Usage_kWh'].max():.2f}")
print(f"Mean: {df['Usage_kWh'].mean():.2f}")
print(f"Median: {df['Usage_kWh'].median():.2f}")

Number of outliers in Usage_kWh: 328 (0.94%)

Usage_kWh statistics:
Min: 0.00
Max: 157.18
Mean: 27.39
Median: 4.57


## Sort Data by Date

In [11]:
# Sort by date to ensure chronological order
df = df.sort_values('date').reset_index(drop=True)

print("Data sorted by date")
print(f"\nFirst 5 dates:")
print(df['date'].head())
print(f"\nLast 5 dates:")
print(df['date'].tail())

Data sorted by date

First 5 dates:
0   2018-01-01 00:00:00
1   2018-01-01 00:15:00
2   2018-01-01 00:30:00
3   2018-01-01 00:45:00
4   2018-01-01 01:00:00
Name: date, dtype: datetime64[ns]

Last 5 dates:
35035   2018-12-31 22:45:00
35036   2018-12-31 23:00:00
35037   2018-12-31 23:15:00
35038   2018-12-31 23:30:00
35039   2018-12-31 23:45:00
Name: date, dtype: datetime64[ns]


## Save Processed Data

In [12]:
# Save the preprocessed data
output_path = '../processed_data/steel_data_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"Preprocessed data saved to: {output_path}")
print(f"Final dataset shape: {df.shape}")

Preprocessed data saved to: ../processed_data/steel_data_cleaned.csv
Final dataset shape: (35040, 11)


## Summary

Data preprocessing completed:
- Dataset loaded successfully
- Date column converted to datetime format
- Data sorted chronologically
- Missing values checked
- Duplicates checked
- Data validation performed
- Cleaned data saved for next steps